# 데이터 증강

In [1]:
import cv2
import numpy as np
import imgaug.augmenters as iaa
import imgaug as ia
from tqdm import tqdm
import time
import os

### 필요한 사용자 지정 함수 먼저 지정

In [2]:
def Yolo2Coord(size, box):
  category = int(box[0]) # class
  x_center, y_center, width, height = float(box[1]), float(box[2]), float(box[3]), float(box[4])
  img_width, img_height = size[0], size[1]
  x_min = int((x_center -width/2.) * img_width)
  x_max = int((x_center + width/2.) * img_width)
  y_min = int((y_center - height/2.) * img_height)
  y_max = int((y_center + height/2.) * img_height)
  return [category, x_min, y_min, x_max, y_max]

In [3]:
# 좌표를 yolo label형식으로 D만들어 주기
def Pascal2Yolo(box,size):
    image_width, image_height = size[0], size[1]
    # xmin, ymin, xmax, ymax
    x_center = ((box[2] + box[0]) / 2) / (image_width)
    y_center = ((box[3] + box[1]) / 2) / (image_height)
    width = (box[2] - box[0]) / (image_width)
    height = (box[3] - box[1]) / (image_height)
    return [str(x_center), str(y_center), str(width), str(height)]

In [4]:
# 증강된 좌표 list 만들어주기
def after_box(bbs):
  before = []
  for i in range(len(bbs)):
    x_min = bbs[i][0][0]
    y_min = bbs[i][0][1]
    x_max = bbs[i][1][0]
    y_max = bbs[i][1][1]
    before.append([x_min,y_min,x_max,y_max])
  return before

#### 본격적인 증강

In [73]:
# 경로 설정
root_path = "C:/Users/jason/Desktop/workspace_for_augmentation/pepper/valid/" 
image_path = "C:/Users/jason/Desktop/workspace_for_augmentation/pepper/valid/images/" # 원천 이미지
label_path = "C:/Users/jason/Desktop/workspace_for_augmentation/pepper/valid/labels/" # 원천 이미지 라벨

images = list(os.listdir(image_path))
print(len(images),images[:5],sep="\n")
labels = list(os.listdir(label_path))
print(len(labels),labels[:5],sep="\n")

377
['V006_79_1_01_01_01_13_1_0149z_20201007_7.jpg', 'V006_79_1_01_01_01_13_1_0424z_20201023_5.jpg', 'V006_79_1_01_01_01_13_1_0868z_20201019_1.jpg', 'V006_79_1_01_01_01_13_1_0868z_20201019_19.jpg', 'V006_79_1_01_01_01_13_1_0868z_20201113_6.jpg']
377
['V006_79_1_01_01_01_13_1_0149z_20201007_7.txt', 'V006_79_1_01_01_01_13_1_0424z_20201023_5.txt', 'V006_79_1_01_01_01_13_1_0868z_20201019_1.txt', 'V006_79_1_01_01_01_13_1_0868z_20201019_19.txt', 'V006_79_1_01_01_01_13_1_0868z_20201113_6.txt']


In [81]:
aug_name = "aug_v3_"

In [82]:
for img in tqdm(images):
  image = cv2.imread(image_path + img) # 이미지 불러오기
  label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
  height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
  image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 바꿔주기 --> 나중에 편하다

  # bounding box 라벨 저장
  label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

  with open(label_path+label,"r") as f:
    labs = f.readlines() 
    for lab in labs:
      label_list.append(lab.split()) # 라벨들을 리스트에 저장
    f.close()

  """
  bounding box label은 category(클래스;병명),x_center, y_center, width, height로 되어 있다
  이를 좌표값 형식(category, x_min, y_min, x_max, y_max)로 바꿔준다.
  Yolo2Coord() : x_center, y_center, width, height --> x_min(좌상단 x), y_min(좌상단 y), x_max(우하단 x), y_max(우하단 y)
  
  """

  new_bounding_boxes = []
  for box in label_list:
    new_bounding_boxes.append(Yolo2Coord(image_size,box))
  cate = str(new_bounding_boxes[0][0]) # 나중에 사용할 예정
  # print(new_bounding_boxes) # [category, x_min, y_min, x_max, y_max]

  # augmentation 할 때 쓰는 BoundingBox 타입으로 만들어주자
  ia_bounding_boxes = []

  for box in new_bounding_boxes:
    ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

  # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None) 이런 형식으로 되어 있음

  # print(ia_bounding_boxes)
  
  # 본격적인 augmentation
  bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes,shape = image.shape)

  seq = iaa.Sequential([
      iaa.AddToHueAndSaturation((-20, 20), per_channel=True),
      iaa.Sometimes(0.5,iaa.Invert(0.1,per_channel=True)),
      iaa.Sometimes(0.2,iaa.Grayscale(alpha=(0.0,0.5))),
      iaa.OneOf([
                  iaa.Flipud(0.3),              # 30%의 이미지는 상하 반전
                  iaa.Fliplr(0.4)]),
      iaa.Sometimes(0.5, iaa.Superpixels(p_replace=(0,0.5),n_segments = (20,100))),
      iaa.Sometimes(0.5,iaa.LinearContrast((0.75, 1.5), per_channel=0.5)),
      iaa.Sometimes(0.5,iaa.Affine(shear=(-4,4), 
          rotate=(-40,40),fit_output=True #  fit_output=True 옵션을 적용해야 회전시켜도 이미지가 잘리지 않는다.
      )),
      iaa.Sometimes(0.5,iaa.Emboss(alpha=(0, 0.5), strength=(0, 1.0))),
      iaa.OneOf([   iaa.Grayscale(alpha=(0.0,0.3)),
                    iaa.MedianBlur(k=(3, 11)),
                ]),
      iaa.Sometimes(0.5,iaa.Sharpen(alpha=(0,0.5), lightness=(0.75,1.5))) 
  ], random_order=True)


  image_aug, bbs_aug = seq(image=image, bounding_boxes = bbs) # image_aug : 증강된 이미지 # bbs_aug : 증강이 적용된 bounding box 좌표

  cv2.imwrite(root_path + "aug_img/"  + aug_name +img, image_aug) # 증강 이미지 저장 경로

  # 증강된 사진 좌표
  resized = image_aug.shape # 좌표화된 bounding box label을 yolo형식으로 바꿔줄 때 필요하다
  rotated_size = [resized[1],resized[0]] # 마찬가지로 height, width -> width, height 형식으로 바꿔준다

  # after_box() : boundingbox 형식으로 묶여있는 x_min, y_min, x_max, y_max 좌표를 리스트로 정리해준다
  augmented_coords = after_box(bbs_aug)

  # Pascal2Yolo(box,size) : yolo형식에 맞는 label로 바꿔준다
  yolos = []
  for i in range(len(augmented_coords)):
    a = Pascal2Yolo(augmented_coords[i],rotated_size)
    final = cate + " " + " ".join(a)
    yolos.append(final)
    # print(final)
  
    # cate --> str(new_bounding_boxes[0]); 카테고리(병명)
  # print(yolos)
  # print("===================================================================================")


  f = open(root_path + "aug_label/" + aug_name +label,"w",encoding = "utf-8", newline='') # 증강 라벨 저장 경로
  for yolo in yolos:
    f.write(yolo)
    f.write("\n")
  f.close()




100%|██████████| 377/377 [03:05<00:00,  2.03it/s]
